# A2C com Stable Baselines3

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap09/cap09-1-A2C-stablebaselines.ipynb)

Vamos usar o algoritmo **A2C** (*Advantage Actor Critic*) neste Google Colab.

Referências:
- Notebook baseado neste tutorial: https://github.com/araffin/rl-tutorial-jnrr19
- Stable-Baselines3: https://github.com/DLR-RM/stable-baselines3
- Documentação: https://stable-baselines3.readthedocs.io/en/master/index.html
- Caso queira usar agentes pré-treinados, veja [RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo)

Alguns textos do tutorial original foram mantidos.

A seguir, instalamos as dependências necessárias.

In [ ]:
import sys
from IPython.display import clear_output

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !apt-get install ffmpeg
    !pip install "stable-baselines3[extra]==2.1.0"
    #clear_output()

In [ ]:
!mkdir log_dir

## Imports

In [ ]:
import gymnasium as gym
import numpy as np
import tensorboard

%load_ext tensorboard

import stable_baselines3
stable_baselines3.__version__

Vamos importar especificamente o algoritmo **A2C**. Você pode importar outros algoritmos, instanciar e usar de forma idêntica ou parecida.

In [ ]:
from stable_baselines3 import A2C

Você também precisa importar a classe que vai representar a **rede neural da política**. É recomendado importar do submódulo específico do algoritmo desejado. No caso, vamos usar uma rede com camadas totalmente conectadas (ou seja, MLP - Multi-Layer Perceptron).

In [ ]:
from stable_baselines3.a2c import MlpPolicy

## 1 - Criar o ambiente e instanciar o agente

Estamos criando o ambiente e passando para a classe **A2C**, que permite testar ou treinar o agente.

Sobre os parâmetros:
- `n_steps`: indica quantos passos vai fazer por atualização
- `ent_coef`: indica o peso do "bônus de exploração" (coeficiente de entropia)
- `tensorboard_log`: diretório para guardar os logs
- mais informações: https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html



In [ ]:
env = gym.make("Acrobot-v1", render_mode="rgb_array")
#env = gym.make("LunarLander-v2", render_mode="rgb_array")

model = A2C(MlpPolicy, env, n_steps=16, ent_coef=0.01, verbose=1, tensorboard_log="log_dir")

Mais alguns detalhes sobre o uso de `MlpPolicy`:
- o tamanho da entrada será deduzido do espaço de *estados do ambiente*
- o número de nós de saída será deduzido do *espaço de ações*

Você também pode passar a string `"MlpPolicy"`, sem precisar importar a classe.

## 2 - Função auxiliar para rodar e avaliar o agente

Na verdade, Stable-Baselines3 já oferece função similar, que poderia ser importada assim:

`from stable_baselines3.common.evaluation import evaluate_policy`

Mas vamos usar uma versão própria:

In [ ]:
def evaluate(model, n_eval_episodes=100, deterministic=True):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single environment
    env = model.get_env()
    all_episode_rewards = []
    for i in range(n_eval_episodes):
        episode_rewards = []
        done = False
        obs = env.reset()
        while not done:
            # _model_state is only useful when using LSTM policies
            action, _model_state = model.predict(obs, deterministic=deterministic)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            # also note that the step only returns a 4-tuple, as the env that is returned
            # by model.get_env() is an sb3 vecenv that wraps the >v0.26 API
            obs, reward, done, info = env.step(action)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    std_episode_rewaard = np.std(all_episode_rewards)
    #print("Reward: ", mean_episode_reward, "+/-", std_episode_rewaard)

    return mean_episode_reward, std_episode_rewaard

Vamos avaliar o agente não-treinado:

In [ ]:
# Random Agent, before training
mean_reward, std_reward = evaluate(model, n_eval_episodes=50)

print(f"Retorno médio: {mean_reward:.2f} +/- {std_reward:.2f}")

## 3 - Treina o agente e avalia

Treinar é bem simples. Note que você precisa informar a quantidade de **passos** (e não de *episódios*).

In [ ]:
# Train the agent for 100 thousand steps
model.learn(total_timesteps=100_000)

In [ ]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate(model, n_eval_episodes=50)

print(f"Retorno médio: {mean_reward:.2f} +/- {std_reward:.2f}")

## 4 - Visualizar treinamento no Tensorboard

In [ ]:
#%tensorboard --logdir log_dir

In [ ]:
tensorboard.notebook.display(height=550)

## 5 - Visualizar Agente

### 5.1 Definições necessárias

Funções necessárias para salvar um vídeo.

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

We will record a video using the [VecVideoRecorder](https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder) wrapper.

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
  # Start the video at step=0 and record the given number of steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, done, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

### 5.2 Grava e exibe vídeo



In [ ]:
record_video("Acrobot-v1", model, video_length=1000, prefix="a2c-acrobot")
#record_video("LunarLander-v2", model, video_length=1000, prefix="a2c-lunar")

In [ ]:
show_videos("videos", prefix="a2c-acrobot")
#show_videos("videos", prefix="a2c-lunar")

## Bônus: Treinar um modelo de RL em UMA linha

Vamos treinar o **DQN**, para comparar com o *A2C*.

In [ ]:
from stable_baselines3 import DQN

Usaremos apenas uma linha de código. A classe da política será inferida e o ambiente será criado automaticamente a partir das strings dadas. Veja mais na [documentação](https://stable-baselines.readthedocs.io/en/master/guide/quickstart.html).

In [ ]:
model2 = DQN('MlpPolicy', "CartPole-v1", buffer_size=20_000, learning_starts=2_000, target_update_interval=4_000, tensorboard_log="log_dir", verbose=1).learn(100_000)

In [ ]:
# faça o código para testar 'model2' aqui